In [1]:
!pip install datasets transformers  accelerate -U tensorflow

  Using cached datasets-2.14.6-py3-none-any.whl (493 kB)
  Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached multiprocess-0.70.15-py310-none-any.whl (134 kB)
  Using cached huggingface_hub-0.18.0-py3-none-any.whl (301 kB)
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Using cached safetensors-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)


In [1]:
from datasets import load_dataset
dataset = load_dataset("imdb")

In [2]:
from transformers import AutoTokenizer
brt_tkn = AutoTokenizer.from_pretrained("bert-base-cased")
def generate_tokens_for_imdb(examples):
 return brt_tkn(examples["text"], padding="max_length", truncation=True)
tkn_datasets = dataset.map(generate_tokens_for_imdb, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
training_dataset = tkn_datasets["train"].shuffle(seed=42).select(range(100))
evaluation_dataset = tkn_datasets["test"].shuffle(seed=42).select(range(100))

In [3]:
from transformers import AutoModelForSequenceClassification
mdl = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="imdb")

In [6]:
import numpy as np
from datasets import load_metric
mdl_metrics = load_metric("accuracy")
def calculate_metrics(eval_pred):
 logits, labels = eval_pred
 predictions = np.argmax(logits, axis=-1)
 return mdl_metrics.compute(predictions=predictions, references=labels)

<ipython-input-6-5e4881016326>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  mdl_metrics = load_metric("accuracy")


In [7]:
from transformers import TrainingArguments, Trainer
from accelerate import Accelerator
accelerator = Accelerator()
mdl.to(accelerator.device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:


trng_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps = 4,
     dataloader_num_workers=accelerator.num_processes,  # Use multiple processes for data loading
    )

In [9]:
Mdl_trainer = Trainer(
    model=mdl,
    args=trng_args,
    train_dataset=training_dataset,
    eval_dataset=evaluation_dataset,
    compute_metrics=calculate_metrics,
)

In [10]:
Mdl_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.683054,0.600000
2,No log,0.584799,0.690000


TrainOutput(global_step=36, training_loss=0.6245303683810763, metrics={'train_runtime': 2626.7628, 'train_samples_per_second': 0.114, 'train_steps_per_second': 0.014, 'total_flos': 75775983943680.0, 'train_loss': 0.6245303683810763, 'epoch': 2.88})

In [11]:
Mdl_trainer.save_model()

In [14]:
metrics = Mdl_trainer.evaluate(evaluation_dataset)

NameError: ignored

In [15]:
Mdl_trainer.log_metrics("eval", metrics)
Mdl_trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       2.88
  eval_accuracy           =       0.69
  eval_loss               =     0.5848
  eval_runtime            = 0:03:39.96
  eval_samples_per_second =      0.455
  eval_steps_per_second   =      0.227


In [4]:
PATH = 'test_trainer/'
md = AutoModelForSequenceClassification.from_pretrained(PATH, local_files_only=True)
def make_classification(text):
 # Tokenize
  inps = brt_tkn(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
  # get output
  outputs = md(**inps)
  # softmax for generating probablities
  probablities = outputs[0].softmax(1)
  # get best match.
  return probablities .argmax()

In [26]:
text = """
It was fun to watch but It did not impress that much I think i
waste my money popcorn time pizza burgers everything.
Akshay should make only comedy movies these King type movies
suits on king like personality of actors Total waste.
"""
print(make_classification(text))

tensor(0)


In [ ]:
from transformers import pipeline

chef = pipeline('text-classification',model=md, tokenizer='anonymous-german-nlp/german-gpt2',config={'max_length':800})

result = chef('Zuerst Tomaten')[0]['generated_text']



In [32]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

In [10]:
from transformers import pipeline

# Load the fine-tuned model (replace 'path/to/your/fine-tuned-model' with the actual path)
model_path = 'test_trainer/'
classifier = pipeline('text-classification', model=model_path, tokenizer='bert-base-cased')


In [12]:
text = """
It was fun to watch but It did not impress that much I think i
waste my money popcorn time pizza burgers everything.
Akshay should make only comedy movies these King type movies
suits on king like personality of actors Total waste.
"""

In [13]:
# Perform text classification inference on a single text
result = classifier(text)

# Print the result
print(result)


[{'label': 'LABEL_0', 'score': 0.5803128480911255}]
